> **This notebook uses langchain agents with a custom tool to solve a math problem.**
> - **Thought:** Tools should be able to extract numerical data from the text input.
> - **Observation:** Results are NOT accurate.

## Environment Setup

In [1]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

In [2]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    #temperature=0.3,
    verbose=True
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    #temperature=0.3,
    verbose=True
)

## Code

In [3]:
prompt_template = """You are an AI assistant that helps the person answer a question based on the data below.
Answer the question below using the data from these records.
Show your work step by step.

# Data
{data}

# Conversation
Question: {question}
Answer:
"""

In [4]:
data = """Name,Gender,Age
John,M,13
Steve,M,25
Sarah,F,18
Jess,F,45
Will,M,50
"""

In [18]:
from langchain.tools import tool  

@tool
def compareValues(comparisonQuestion: str) -> bool:
    '''
        input string comparisonQuestion is a question in natural language about comparing two numerical values.
        For example, a question could be:
        "Is 5 greater than 3?"
    '''

    prompt = "Respond with Yes or No. " + comparisonQuestion
    result = llm(prompt)
    
    return result

In [19]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain

#from langchain.agents import load_tools
#tools = load_tools(["llm-math"], llm=llm)


llm_math_chain = LLMMathChain.from_llm(llm=chat, verbose=True)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions using math."
    ),
    Tool(
        name="CompareValues",
        func=compareValues,
        description="Useful to comparing two numerical values. Question should be in the form of 'Is 10 greater than 20?', 'Is 10 less than 20?', or 'Is 10 equal to 20?'"
    )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

prompt = prompt_template.format(
    data=data,
    question="Who are older than 30 years old?"
)

agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to compare ages to see who is older than 30.
Action: CompareValues
Action Input: Is 18 greater than 30?
Observation: 

No
Thought: I need to check the next record.
Action: CompareValues
Action Input: Is 25 greater than 30?
Observation: 

No
Thought: I need to check the next record.
Action: CompareValues
Action Input: Is 45 greater than 30?
Observation: 

Yes
Thought: I need to check the next record.
Action: CompareValues
Action Input: Is 50 greater than 30?
Observation: 

Yes
Thought: I now know the final answer.
Final Answer: John, Steve, Sarah, and Will are older than 30 years old.

> Finished chain.


'John, Steve, Sarah, and Will are older than 30 years old.'